# Лабораторная работа 2
# Тема 2. Генерация простых чисел

# <font color='red'>Разбираю теоретическую  часть</font>

## Тестирование чисел на простоту

### Метод пробных делений

#### Алгоритм

**Метод пробных делений.**  
Input: $n\in\mathbb{N}.$  
Output: $n$ -- простое?    
1$.\;\; d:=2$  
2$.\;$ While $\;d\leq \sqrt n$   
$\qquad$ If $\;n = 0 \mod d$  
$\qquad\qquad$ Return False   
$\qquad$  $d:=d+1$    
3$.\;$ Return True

#### Реализация

In [2]:
def TrialDivision(n):
    # counter = 0
    d = 2
    while d <= n**0.5:
        if n % d == 0:
            return False
        d += 1
        # counter += 1
    # return True, counter
    return True

n = 43
TrialDivision(n)

True

#### Кол-во арифметических операций и временная сложность

Если $n$ -- простое, то количество арифметических операций, выполняемых во время работы алгоритма, равно  $$f(n)=\left[\sqrt n\right]-1=O\left(n^{1/2}\right).$$

Поэтому для временной сложности алгоритма справедлива оценка
$$
  T(N):=\max\limits_{\left<n\right>\leq N} f(n)=O\left(2^{N/2}\right).
$$
Метод пробных делений прост для понимания и реализации, но экспоненциален.

#### Замечания

Вопрос: почему количество операций будет $\left[\sqrt n\right] - 1?$

Ответ: потому что проверка на простоту начинается не с единицы, а с двойки $d := 2$.

Вопрос: почему алгоритм должен работать пока $d \le \sqrt n?$

Ответ: потому что после $\sqrt n$ будут встречаться те же самые множители числа $n$, что и до корня. А если множителей не нашлось, то и после корна их тем более не будет.

### Проверка теорем

#### Кольцо вычетов из предыдущей лабораторной (но с ошибками при делении и разности)

In [4]:
class Ring:
    def __init__(self, remainder, m):
        self.remainder = remainder
        self.mod = m

    def __repr__(self):
        return "(%s)" % (self.remainder)
    
    def __add__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder + other.remainder) % self.mod, self.mod)
        
    def __sub__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder - other.remainder) % self.mod, self.mod)
        
    def __mul__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder * other.remainder) % self.mod, self.mod)
        
    def __truediv__(self, other):
        if isinstance(other, Ring):
            return Ring((self.remainder / other.remainder) % self.mod, self.mod)
        
    def __pow__(self, num):
        return Ring(NewFastPower(self.remainder, num, self.mod), self.mod)
        
    def __neg__(self):
        return Ring(reverse_elem(self.remainder, self.mod), self.mod)
        
    def __eq__(self, other):
        if isinstance(other, Ring):
            if self.remainder == other.remainder:
                return True
            else:
                return False
            
def reverse_elem(a, b):
    if a == b - 1:
        return a % b
    a_copy = a
    b_copy = b
    m11, m12 = 1, 0
    m21, m22 = 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        m11, m12 = m12, m11 - m12*q
        m21, m22 = m22, m21 - m22*q
    gcd = m11 * a_copy + m21 * b_copy
    if gcd == 1:
        return m11
    else:
        raise ValueError('base is not invertible for the given modulus')
        
# если я учитываю, что a - целое и, что есть проблема с единицей, то
def NewFastPower(a, d, m):
    b = abs(a)
    if d == 1:
        return b % m
    bin_lst = [int(i) for i in bin(d)[3:]]
    for elem in bin_lst:
        b = (b ** 2) * (a ** elem) % m
    return b

#### Проверяю выполнение теоремы Эйлера на конкретном примере

##### Мультипликативная група кольца вычетов по модулю n

Пример из википедии: https://ru.wikipedia.org/wiki/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%BF%D0%BB%D0%B8%D0%BA%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%B0_%D0%BA%D0%BE%D0%BB%D1%8C%D1%86%D0%B0_%D0%B2%D1%8B%D1%87%D0%B5%D1%82%D0%BE%D0%B2

In [5]:
n = 42
Z_star_n = [Ring(1, 42), Ring(5, 42), Ring(11, 42), Ring(13, 42), Ring(17, 42), Ring(19, 42), Ring(23, 42), Ring(25, 42), Ring(29, 42), Ring(31, 42), Ring(37, 42), Ring(41, 42)]
Z_star_n

[(1), (5), (11), (13), (17), (19), (23), (25), (29), (31), (37), (41)]

##### Выполнение теоремы

**Теорема Эйлера**  
 Пусть $n$ натуральное число, не равное $1.$ Если
   ${a\in\mathbb{Z}_n^*,}$ то
    $$
      {a^{\varphi(n)}\equiv1\mod n.}
    $$

In [102]:
import sympy # для использования реализации функйии Эйлера sympy.totient(n)
import random
main_natural_num = 100
n1 = random.randint(1, main_natural_num)
a = Ring((Z_star_n[random.randint(0, len(Z_star_n)) - 1]).remainder, n1)
# a = Z_star_n[random.randint(0, len(Z_star_n)) - 1]
print(n1, a)
((a ** sympy.totient(n1)).remainder % n1) == (1 % n1)

3 (23)


True